Reference - this code is just modification of https://github.com/dennybritz/reinforcement-learning

In [1]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

In [2]:
import gym
import itertools
import matplotlib
import sys
import tensorflow as tf
import collections
from copy import deepcopy

matplotlib.style.use('ggplot')

# Load Data

In [4]:
dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataLabeled\\poloniex\\5min'
fileName = 'USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
file = '{dataRoot}\\{fileName}'.format(dataRoot=dataRoot, fileName=fileName)

usdtBtc = pd.read_csv(file, sep='\t').query("date >= '2017-01-01'")\
    .sort_values('timestamp').reset_index().drop('index', axis=1)
usdtBtc = usdtBtc

trainRatio = 0.7
trainSize = int(usdtBtc.shape[0] * trainRatio)
usdtBtcTrain = usdtBtc.iloc[:trainSize]
usdtBtcTest = usdtBtc.iloc[trainSize:].reset_index().drop('index', axis=1)

usdtBtcManagerTrain = CurrencyDataManager(usdtBtcTrain['close'], usdtBtcTrain)
usdtBtcManagerTest = CurrencyDataManager(usdtBtcTest['close'], usdtBtcTest)
wallet = {'usdt': 1000}
contractPair = ContractPair.new('usdt', 'btc')

# Slicing Array

In [ ]:
%%timeit
for i in range(1000):
    usdtBtc.iloc[10:11, :]

In [15]:
%%timeit
for i in range(1000):
    usdtBtc.ix[10:11, :]

1 loop, best of 3: 276 ms per loop


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:258: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [14]:
%%timeit
arr = usdtBtc.values
for i in range(1000):
    arr[10:11, :]

1 loop, best of 3: 121 ms per loop


# Data Classes

In [5]:
from trading.money.transaction import SellTransaction
from trading.money.contract import ContractPair
from collections import namedtuple

In [6]:
%%timeit
cPair = ContractPair('usdt', 'btc')

The slowest run took 8.38 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 393 ns per loop


In [7]:
ContractPairNT = namedtuple('ContractPairNT', ['priceContract', 'tradeContract'])

In [8]:
%%timeit
tmp = ContractPairNT('usdt', 'btc')

The slowest run took 6.69 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 438 ns per loop


In [16]:
priceContract = 'usdt'
tradeContract = 'btc'
def contractPair(priceContract, tradeContract):
    return {'priceContract': priceContract, 'tradeContract': tradeContract}

In [17]:
%%timeit
tmp = contractPair(priceContract, tradeContract)

The slowest run took 5.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 206 ns per loop


In [18]:
%%timeit
tmp = {'priceContract': priceContract, 'tradeContract': tradeContract}

10000000 loops, best of 3: 96.6 ns per loop


In [20]:
class ContractPairTmp:
    @staticmethod
    def contractPair(priceContract, tradeContract):
        return {'priceContract': priceContract, 'tradeContract': tradeContract} 

In [21]:
%%timeit
tmp = ContractPairTmp.contractPair(priceContract, tradeContract)

The slowest run took 8.85 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 207 ns per loop


In [ ]:
tmp = Contract.new(name, value)

# Accesing object attribute vs dictionary key

In [1]:
class O:
    def __init__(self):
        self.contractPair = "usdt-btc"
        self.timestamp = 1
        self.amount = 10.
        self.price = 2.
        self.fee = .5

d = {'contractPair': "usdt-btc", 'timestamp': 1, 'amount': 10., 'price': 2., 'fee': .5}
o = O()

In [6]:
%%timeit
a = o.timestamp

The slowest run took 16.95 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 68.1 ns per loop


In [7]:
%%timeit
a = d['timestamp']

The slowest run took 19.77 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 58.4 ns per loop


# Currency Data Manager

In [65]:
class DataManager:

    def tick(self, historyLen):
        raise NotImplementedError()

class CurrencyDataManager2(DataManager):

    # expected prices to be pandas dataframe with columns: timestamp, open, close, high, low, volume
    def __init__(self, prices, data):
        self.prices = prices.values
        self.maxTime = self.prices.shape[0]
        self.data = data.values
        self.time = 1

    def tick(self):
        if self.time >= self.maxTime:
            self.time = 1
#             return None, None
        time = self.time
        price = self.prices[time]
        first = time - 1
        history = self.data[first, :]

        self.time += 1
        return history, price

    def has_tick(self):
        return self.time < self.maxTime
    
manager = CurrencyDataManager2(usdtBtcTrain['close'], usdtBtcTrain)

In [68]:
%%timeit
manager.tick()

The slowest run took 12.98 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 791 ns per loop


In [60]:
import cProfile, pstats, io
pr = cProfile.Profile()
pr.enable()

for i in range(100000):
    manager.tick(1)

pr.disable()

In [61]:
s = io.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         100019 function calls in 0.146 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.104    0.000    0.104    0.000 <ipython-input-47-91de9d08af8b>:15(tick)
        1    0.042    0.042    0.145    0.145 <ipython-input-60-54b01f2de953>:5(<module>)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.145    0.073 C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2851(run_code)
        2    0.000    0.000    0.000    0.000 C:\ProgramData\Anaconda3\lib\codeop.py:132(__call__)
        1    0.000    0.000    0.000    0.000 <ipython-input-60-54b01f2de953>:8(<module>)
        2    0.000    0.000    0.145    0.073 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\hooks.py:139(__call__)
        2    0.000    0.000    0.000    0.000 C:\ProgramData\A

In [ ]:
def filterEmpty(ll):
    return list(filter(lambda x: len(x)!=0, ll))

def statsToPandas(stats):
    lines = filterEmpty(stats.getvalue().split('\n'))
    cols = filterEmpty(lines[2].split(' '))
    rows = [filterEmpty(l.split(' ')) for l in lines[3:]]
    rows = [l[:5] + [''.join(l[5:])] for l in rows]
    df = pd.DataFrame(rows, columns=cols)
    return df

In [ ]:
df = statsToPandas(s)
kryplCalls = df[df['filename:lineno(function)'].str.contains('krypl-project')]
kryplCalls['file'] = kryplCalls['filename:lineno(function)'].apply(lambda x: x.replace('D:\\Gitprojekty\\krypl-project\\', ''))\
    .apply(lambda x: x.replace('D:\\GitProjects\\krypl-project\\', ''))
kryplCalls.sort_values('tottime', ascending=False)